In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow==2.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 54.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: goo

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# Set model parameters and paths
input_shape = (128, 128, 1)
model_path = '/content/drive/MyDrive/FYP/lung_segmentation_unet .h5'
covid_data_dir = '/content/drive/MyDrive/FYP/Dataset/Normal'  # Path to COVID images folder
output_dir = '/content/drive/MyDrive/FYP/Dataset/Normal_Masks'  # Directory to save the predicted masks

In [ ]:
# Load the pre-trained UNet model
from tensorflow.keras.models import load_model

# Load the pre-trained model in .h5 format
model_path_h5 = '/content/drive/MyDrive/FYP/lung_segmentation_unet .h5'  # Path to your .h5 model
saved_model_dir = '/content/drive/MyDrive/FYP/saved_model'  # Directory to save the new SavedModel

# Load the .h5 model
model = load_model(model_path_h5, compile=False)

# Save the model in SavedModel format
model.save(saved_model_dir)
print(f"Model saved in SavedModel format at {saved_model_dir}")

Model saved in SavedModel format at /content/drive/MyDrive/FYP/saved_model


In [ ]:
# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Loop through each image in the COVID folder, segment, and save the masks
for filename in os.listdir(covid_data_dir):
    # Load and preprocess the image
    img_path = os.path.join(covid_data_dir, filename)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (128, 128))  # Resize to match model input shape
    img_normalized = img_resized.astype('float32') / 255.0  # Normalize to [0, 1]
    img_input = np.expand_dims(img_normalized, axis=(0, -1))  # Add batch and channel dimensions

    # Predict the segmentation mask
    predicted_mask = model.predict(img_input)[0, :, :, 0]  # Remove batch dimension

    # Threshold the mask to binary (0, 1)
    thresholded_mask = (predicted_mask > 0.5).astype(np.uint8) * 255  # Convert to binary mask for visualization/storage

    # Resize mask back to original image size if necessary
    mask_resized = cv2.resize(thresholded_mask, (img.shape[1], img.shape[0]))

    # Save the mask
    output_path = os.path.join(output_dir, filename)
    cv2.imwrite(output_path, mask_resized)
    print(f"Saved mask to {output_path}")

print("Mask generation for the COVID dataset is complete.")

Saved mask to /content/drive/MyDrive/FYP/Dataset/Normal_Masks/Normal- (3584).jpg
Saved mask to /content/drive/MyDrive/FYP/Dataset/Normal_Masks/Normal- (3583).jpg
Saved mask to /content/drive/MyDrive/FYP/Dataset/Normal_Masks/Normal- (3578).jpg
Saved mask to /content/drive/MyDrive/FYP/Dataset/Normal_Masks/Normal- (3582).jpg
Saved mask to /content/drive/MyDrive/FYP/Dataset/Normal_Masks/Normal- (3606).jpg
Saved mask to /content/drive/MyDrive/FYP/Dataset/Normal_Masks/Normal- (3602).jpg
Saved mask to /content/drive/MyDrive/FYP/Dataset/Normal_Masks/Normal- (3581).jpg
Saved mask to /content/drive/MyDrive/FYP/Dataset/Normal_Masks/Normal- (3592).jpg
Saved mask to /content/drive/MyDrive/FYP/Dataset/Normal_Masks/Normal- (3575).jpg
Saved mask to /content/drive/MyDrive/FYP/Dataset/Normal_Masks/Normal- (360).jpg
Saved mask to /content/drive/MyDrive/FYP/Dataset/Normal_Masks/Normal- (3590).jpg
Saved mask to /content/drive/MyDrive/FYP/Dataset/Normal_Masks/Normal- (3594).jpg
Saved mask to /content/drive/

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os

# Paths to your images and masks
covid_data_dir = '/content/drive/MyDrive/FYP/Dataset/Normal'  # Original images
output_dir = '/content/drive/MyDrive/FYP/Dataset/Normal_Masks'  # Generated masks

def visualize_predictions(image_dir, mask_dir, num_samples=5):
    images = sorted(os.listdir(image_dir))[:num_samples]

    for filename in images:
        img_path = os.path.join(image_dir, filename)
        mask_path = os.path.join(mask_dir, filename)

        # Load images and masks for visualization
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        pred_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        plt.figure(figsize=(10, 5))

        # Display original image
        plt.subplot(1, 2, 1)
        plt.imshow(img, cmap='gray')
        plt.title('Original Image')

        # Display generated mask
        plt.subplot(1, 2, 2)
        plt.imshow(pred_mask, cmap='gray')
        plt.title('Generated Mask')

        plt.show()

# Visualize a few predictions
visualize_predictions(covid_data_dir, output_dir, num_samples=20)

Output hidden; open in https://colab.research.google.com to view.